In [1]:
import sys
sys.path.append('/home/fernando/Proyectos/PÓS-GRADUAÇÂO/TESIS-DOUTORADO-2/PESQUISA/tools/cnn_patient_people/library');

In [2]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf

# Variable globales

In [3]:
## Seed for the random variables
seed_number=0;

## Dataset 
dataset_base_test_dir= '/mnt/boveda/DATASETs/PATIENT-IMAGES/patient_people/test';


## Training hyperparameters
BATCH_SIZE=32;

## Model of network
#model_type = 'efficientnet_b3'
model_type = 'mobilenet_v3';
#model_type = 'inception_v3';
#model_type = 'inception_resnet_v2';
#model_type = 'resnet_v2_50';

times=10;


## Output
output_base_dir = '/mnt/boveda/DOCTORADO2/cnn_patient_people/test';



# If command line

In [4]:
#print('cmd entry:', sys.argv)

for n in range(len(sys.argv)):
    if sys.argv[n]=='--model':
        model_type=sys.argv[n+1];
    if sys.argv[n]=='--times':
        times=int(sys.argv[n+1]);


best_model_file='/home/fernando/Downloads/TESIS-DOUTORADO-2/PESQUISA/tools/cnn_patient_people/library/models';
best_model_file=os.path.join(best_model_file,'modelo_'+model_type+'.h5');

print('model_type:',model_type)
print('times:',times)

model_type: mobilenet_v3
times: 10


# Set seed of random variables


In [5]:
np.random.seed(seed_number)
tf.keras.utils.set_random_seed(seed_number);

# Data augmentation configuration

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

idg_test= ImageDataGenerator(rescale=1./255 )


# Creating output directory

In [7]:
#output_dir = os.path.join(output_base_dir,'holdout');
output_dir = os.path.join(output_base_dir,model_type);
try: 
    os.mkdir(output_base_dir) 
except: 
    pass

try: 
    os.mkdir(output_dir) 
except: 
    pass


# Create new model

In [8]:
import lib_model as mpp

model, target_size = mpp.create_model('',model_type=model_type);
model.summary()

mpp.save_model_parameters(model, os.path.join(output_dir,'parameters_stats.m'));


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1024)              1529968   
                                                                 
 dense (Dense)               (None, 2)                 2050      
                                                                 
Total params: 1,532,018
Trainable params: 2,050
Non-trainable params: 1,529,968
_________________________________________________________________


# Defining directories

In [9]:

test_data_generator  = idg_test.flow_from_directory(dataset_base_test_dir, 
                                                    target_size=target_size,
                                                    batch_size=BATCH_SIZE,
                                                    shuffle=True
                                                    );

Found 273 images belonging to 2 classes.


# Compile


In [10]:
# COMPILE NEW MODEL
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


# Time

In [11]:
from pytictoc import TicToc
t = TicToc() 

# Evaluate best model

In [12]:
# LOAD BEST MODEL to evaluate the performance of the model
model.load_weights(best_model_file);


L=test_data_generator.samples;
print('L:',L);

t.tic();
for m in range(times):
    results = model.evaluate(test_data_generator,verbose=0)
t0=t.tocvalue();

results = dict(zip(model.metrics_names,results))
print(results,"\n\n");


with open(os.path.join(output_dir,"results_testing.m"), 'w') as f: 
    for key, value in results.items(): 
        f.write('%s=%s;\n' % (key, value));
    f.write('delayms=%s;\n' % (t0*1000.0/(times*L)));

tf.keras.backend.clear_session()


L: 273
{'loss': 0.14216236770153046, 'accuracy': 0.9633699655532837} 


